In [ ]:
import requests
import pandas as pd
import json
import configparser
from pymongo import MongoClient
from datetime import datetime , date

In [ ]:
def read_key():
    config_reader = configparser.ConfigParser()
    config_reader.read('config_file.ini')
    api_key  = config_reader['Dev']['key']
    return api_key

In [ ]:
def weather_forecat_api():
    url = "https://data.tmd.go.th/nwpapi/v1/forecast/location/daily/place"
    date_now = str(datetime.now().date())
    api_key = read_key()
    querystring = {"province":u"กรุงเทพมหานคร","subarea":"1" ,"fields":"cond,ws10m,rain,tc_min,tc_max,rh", "date":f"{date_now}", "hour":"8", "duration":"14"}
    headers = {
        'accept': "application/json",
        'authorization': f"Bearer {api_key}",
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    return data

In [ ]:
def normalized_json(data):
    forcast_list =  []
    for forcast in data['WeatherForecasts']:
        location = forcast['location']
        for day in forcast['forecasts']:
            combined_data = {**location, **day['data'], 'time': day['time']}
            forcast_list.append(combined_data)
    return forcast_list

def transform_cond(status):
    if status ==1:
        return 'Clear'
    elif status ==2:
        return 'Partly cloudy'
    elif status ==3:
        return 'Cloudy'
    elif  status ==4:
        return 'Overcast'
    elif status ==5:
        return 'Light rain'
    elif status ==6:
        return 'Moderate rain'
    elif status ==7:
        return 'Heavy rain'
    elif status ==8:
        return 'Thunderstorm'
    elif status ==9:
        return 'Very cold'
    elif status ==10:
        return 'Cold'
    elif status ==11:
        return 'Cool'
    elif status ==12:
        return 'Very hot'
    else:
        return None

In [ ]:
def transfrom_data(forcast_list):
    df = pd.DataFrame(forcast_list)
    #filter only areatype == amphoe 
    df_amphoe= df[~((df['areatype']=='province')&(df['amphoe'].isnull()))]
    df_amphoe.drop(columns='tambon',inplace=True)
    #transform status weather
    df_amphoe['cond'] = df_amphoe['cond'].apply(transform_cond)
    df_amphoe.reset_index(inplace=True)
    df_amphoe.drop(columns='index',inplace=True)
    return df_amphoe

In [ ]:
def ingest_2MongoDB(df_amphoe):
    database = myclient['mydb']
    collection = database['weather_forecast']
    data_dict = df_amphoe.to_dict(orient='records')
    collection.insert_many(data_dict)

In [ ]:
if __name__=="__main__":
    
    conn_str = 
    myclient = MongoClient(conn_str)
    
    data = weather_forecat_api()
    forcast_list = normalized_json(data)
    df_amphoe = transfrom_data(forcast_list)
    ingest_2MongoDB(df_amphoe)
    
    myclient.close()